## Run the prior ensemble through the model

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu
plt.rcParams['font.size'] = 12

In [ ]:
t_d = "freyberg6_template"
assert os.path.exists(t_d)," need to run the setup_pestpp_interface notebook first!"

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"freyberg6.pst"))

In [ ]:
pst.pestpp_options["ies_num_reals"] = 10  # restrict number of realisations for binder demo (up to 300 realisation in ensemble)

In [ ]:
pst.control_data.noptmax = -1
pst_filename = "freyberg6_pmc.pst"
pst.write(os.path.join(t_d,pst_filename),version=2)

In [ ]:
pyemu.os_utils.start_workers(
    t_d,
    "pestpp-ies",
    pst_filename,
    num_workers=2,  # restrict number of parallel workers for binder demo to avoid throttling
    master_dir="master_pmc", 
    worker_root='.'
)

In [ ]:
obs_filename = pst_filename.replace(".pst",".0.obs.csv")
obs_df = pd.read_csv(os.path.join("master_pmc",obs_filename),index_col=0)

In [ ]:
obs_df

In [ ]:
pst.obs_groups

In [ ]:
pst.try_parse_name_metadata()
#pst.observation_data.loc[:,"time"] = pst.observation_data.time.apply(np.float)
obs = pst.observation_data
for obs_group in pst.obs_groups:
    obs_g = obs.loc[obs.obgnme==obs_group,:].copy()
    obs_g.sort_values(by="datetime",inplace=True)
    obs_g_df = obs_df.loc[:,obs_g.obsnme]
    fig,ax = plt.subplots(1,1,figsize=(15,2))
    [ax.plot(obs_g.datetime,obs_g_df.loc[i,obs_g.obsnme],color='0.5',alpha=0.5,lw=0.2) for i in obs_g_df.index]
    ax.set_title(obs_group)
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=45, horizontalalignment='right' )
    plt.show()
    
    

# BOOM!!!